In [7]:
import numpy as np

In [ ]:
class Node:
    def __init__(self, x, s = None):
        self.symbol = s
        self.frequency = x
        self.code = None
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.frequency < other.frequency
    
    def __eq__(self, other):

        if isinstance(other, Node):
            return self.frequency == other.frequency
        return False

class CustomList:
    def __init__(self):
        self.data = []

    def pop(self):
        if not self.data:
            raise IndexError("pop from empty list")
        return self.data.pop(0)

    def insert_sorted(self, node):
        for i in range(len(self.data)):
            if self.data[i] > node or self.data[i] == node:
                self.data.insert(i, node)
                return
        self.data.append(node)

    def __len__(self):
        return len(self.data)

def preOrder(root, codebook, codeword):
    if root is None:
        return

    if root.left is None and root.right is None:
        #ans.append(curr)
        codebook[root.symbol] = {"frequency": root.frequency, "code": codeword}
        return

    preOrder(root.left, codebook, codeword + '1')
    preOrder(root.right, codebook, codeword + '0')

def huffmanCodes(s, freq):
    n = len(s)

    custom_list = CustomList()
    
    for i in range(n):
        tmp = Node(freq[i], i)
        custom_list.insert_sorted(tmp)
    
    while len(custom_list) >= 2:
        l = custom_list.pop()
        r = custom_list.pop()

        newNode = Node(l.frequency + r.frequency)
        newNode.left = l
        newNode.right = r

        custom_list.insert_sorted(newNode)

    # Raiz da árvore
    root = custom_list.pop()
    codebook = {}
    preOrder(root, codebook, "")
    return codebook, root

def print_huffman_tree(node, level=0):
    """
    Imprime a árvore de Huffman mostrando os nós e suas frequências.

    Parâmetros:
    ----------
    node : Node
        Nó atual da árvore.
    level : int
        Nível atual na árvore (usado para indentação).
    """
    if node is not None:
        # Imprime o nó atual com indentação baseada no nível
        print("  " * level + f"{level} Frequência: {node.frequency}")
        
        # Recursivamente imprime os filhos (esquerda e direita)
        print_huffman_tree(node.left, level + 1)
        print_huffman_tree(node.right, level + 1)

#### Exemplo 1 (Cover)

In [9]:
#exemplo 1 - cover
s = range(0,5)
freq = [0.25, 0.25, 0.2, 0.15, 0.15] 
codebook, huffman_tree_root = huffmanCodes(s, freq)
for key in sorted(codebook.keys()):
    print(key, codebook[key])
print("\n-----------------------\n")
print_huffman_tree(huffman_tree_root)

0 {'frequency': 0.25, 'code': '01'}
1 {'frequency': 0.25, 'code': '10'}
2 {'frequency': 0.2, 'code': '11'}
3 {'frequency': 0.15, 'code': '000'}
4 {'frequency': 0.15, 'code': '001'}

-----------------------

0 Frequência: 1.0
  1 Frequência: 0.45
    2 Frequência: 0.2
    2 Frequência: 0.25
  1 Frequência: 0.55
    2 Frequência: 0.25
    2 Frequência: 0.3
      3 Frequência: 0.15
      3 Frequência: 0.15


#### Exemplo Huffman Shapping - Artigo do Ungerboeck 

In [10]:
# exemplo com dados do artigo

p_i = np.array([
    0.03872, 0.02991, 0.02991, 0.02311, 0.01785, 0.01785,
    0.01379, 0.01379, 0.00823, 0.00823, 0.00823, 0.00636, 
    0.00636, 0.00379, 0.00379, 0.00293, 0.00293, 0.00226, 
    0.00226, 0.00175, 0.00135, 0.00135, 0.00081, 0.00081, 
    0.00062, 0.00062, 0.00062, 0.00062, 0.00037, 0.00037, 
    0.00022, 0.00017
])

p_i_n = np.array([
    0.03125, 0.03125, 0.03125, 0.03125,
    0.01563, 0.01563, 0.01563, 0.01563,
    0.00781, 0.00781, 0.00781, 0.00781,
    0.00781, 0.00391, 0.00391, 0.00195, 
    0.00195, 0.00195, 0.00195, 0.00195,
    0.00098, 0.00098, 0.00049, 0.00049, 
    0.00049, 0.00049, 0.00049, 0.00049,
    0.00024, 0.00024, 0.00024, 0.00024
])

#comprimento das palavras de código
li = np.round(-np.log2(p_i_n))


s = range(0,32)
freq = p_i_n * 4
# s = range(0,128)
# freq = np.tile(p_i_n, 4) 
codebook, huffman_tree_root = huffmanCodes(s, freq)
for key in sorted(codebook.keys()):
    print(key, codebook[key])
np.sum(p_i_n * 4)
first_quadrant = {k: v for k, v in codebook.items() if 0 <= k <= 31}
for key in sorted(first_quadrant.keys()):
    print(key, first_quadrant[key])


0 {'frequency': 0.125, 'code': '100'}
1 {'frequency': 0.125, 'code': '101'}
2 {'frequency': 0.125, 'code': '010'}
3 {'frequency': 0.125, 'code': '011'}
4 {'frequency': 0.06252, 'code': '0001'}
5 {'frequency': 0.06252, 'code': '0010'}
6 {'frequency': 0.06252, 'code': '0011'}
7 {'frequency': 0.06252, 'code': '1100'}
8 {'frequency': 0.03124, 'code': '00001'}
9 {'frequency': 0.03124, 'code': '11100'}
10 {'frequency': 0.03124, 'code': '11101'}
11 {'frequency': 0.03124, 'code': '11010'}
12 {'frequency': 0.03124, 'code': '11011'}
13 {'frequency': 0.01564, 'code': '000001'}
14 {'frequency': 0.01564, 'code': '111100'}
15 {'frequency': 0.0078, 'code': '1111100'}
16 {'frequency': 0.0078, 'code': '1111101'}
17 {'frequency': 0.0078, 'code': '1111010'}
18 {'frequency': 0.0078, 'code': '1111011'}
19 {'frequency': 0.0078, 'code': '1111110'}
20 {'frequency': 0.00392, 'code': '00000010'}
21 {'frequency': 0.00392, 'code': '00000011'}
22 {'frequency': 0.00196, 'code': '111111100'}
23 {'frequency': 0.00196

#### Exemplo 3

In [11]:
# exemplo 3
freq = np.array([1/16,1/16,1/16,1/16,1/8,1/8, 1/8,1/8,1/4])
s = range(0,9)
codes, huffman_tree_root = huffmanCodes(s, freq)
for key in sorted(codes.keys()):
    print(key, codes[key])

print("\n-----------------------\n")
print_huffman_tree(huffman_tree_root)

0 {'frequency': 0.0625, 'code': '1110'}
1 {'frequency': 0.0625, 'code': '1111'}
2 {'frequency': 0.0625, 'code': '1100'}
3 {'frequency': 0.0625, 'code': '1101'}
4 {'frequency': 0.125, 'code': '010'}
5 {'frequency': 0.125, 'code': '011'}
6 {'frequency': 0.125, 'code': '000'}
7 {'frequency': 0.125, 'code': '001'}
8 {'frequency': 0.25, 'code': '10'}

-----------------------

0 Frequência: 1.0
  1 Frequência: 0.5
    2 Frequência: 0.25
      3 Frequência: 0.125
        4 Frequência: 0.0625
        4 Frequência: 0.0625
      3 Frequência: 0.125
        4 Frequência: 0.0625
        4 Frequência: 0.0625
    2 Frequência: 0.25
  1 Frequência: 0.5
    2 Frequência: 0.25
      3 Frequência: 0.125
      3 Frequência: 0.125
    2 Frequência: 0.25
      3 Frequência: 0.125
      3 Frequência: 0.125


In [12]:
codes

{1: {'frequency': 0.0625, 'code': '1111'},
 0: {'frequency': 0.0625, 'code': '1110'},
 3: {'frequency': 0.0625, 'code': '1101'},
 2: {'frequency': 0.0625, 'code': '1100'},
 8: {'frequency': 0.25, 'code': '10'},
 5: {'frequency': 0.125, 'code': '011'},
 4: {'frequency': 0.125, 'code': '010'},
 7: {'frequency': 0.125, 'code': '001'},
 6: {'frequency': 0.125, 'code': '000'}}